In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
from scipy.stats import norm
import scipy.integrate as integrate

import gym
from gym import spaces

import random
import itertools as it
from itertools import product
from joblib import Parallel, delayed
from toolz import memoize
from contracts import contract
from collections import namedtuple, defaultdict, deque, Counter

import warnings
warnings.filterwarnings("ignore", 
                        message="The objective has been evaluated at this point before.")

from agents import Agent
from oldmouselab import OldMouselabEnv
from policies import FixedPlanPolicy, LiederPolicy
from evaluation import *
from omdc_util import *
from distributions import cmax, smax, sample, expectation, Normal, TruncatedNormal, PointMass, SampleDist, Normal, Categorical

In [5]:
import ipyparallel as ipp 
rc = ipp.Client(profile='default', cluster_id='')
ipp.register_joblib_backend()

In [6]:
def hd_dist(attributes):
    dist = [1,]*attributes
    dist[0] = np.random.randint(85,97)
    for i in range(1,attributes-1):
        dist[i] += np.random.randint(0,100-np.sum(dist))
    dist[-1] += 100-np.sum(dist)
    dist = np.around(np.array(dist)/100,decimals=2)
    np.random.shuffle(dist)
    return dist

def ld_dist(attributes):
    constrain = True
    while constrain:
        dist = [np.random.randint(10,50) for _ in range(attributes)]
        dist = np.around(np.array(dist)/sum(dist),decimals=2)
        constrain = np.min(dist) <= 0.10 or np.max(dist) >= 0.40
    np.random.shuffle(dist)
    return dist

In [7]:
gambles = 7
attributes = 4
high_stakes = TruncatedNormal((9.99+0.01)/2, 0.3*(9.99-0.01),0.01,9.99)
low_stakes = TruncatedNormal((0.25+0.01)/2, 0.3*(0.25-0.01),0.01,0.25)
reward = high_stakes
cost=.03

#set to 20 for sanity check
n_train = 20
n_test = 20

train_envs_hd = [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_train)]
train_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_train)]
train_envs = train_envs_hd+train_envs_ld 

test_envs_hd =  [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_train)]
test_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_train)]
test_envs = test_envs_hd+test_envs_ld 

term_action = train_envs[0].term_action

In [8]:
bo_pol_theta = np.load('data/om_bmps_pols/best/hs_hd_1cents.npy')
bo_pol = LiederPolicy(list(bo_pol_theta))

In [9]:
agent = Agent()
def run_env(policy, env):
    agent.register(env)
    agent.register(policy)
    tr = agent.run_episode()
#     print(tr)
    return {'util': tr['return'], 'actions': tr['actions'],
            'observations': len(tr['actions']) - 1, 'ground_truth': env.ground_truth}

def action_coordinate(env, action):
    return (action//env.outcomes,action%env.outcomes)

def p_grid(env, actions):
    grid = np.zeros((env.gambles+1,env.outcomes))
    grid[0,:] = env.dist
    for i in range(len(actions[:-1])):
        gamble, outcome = action_coordinate(env,actions[i]) 
        grid[gamble+1, outcome] = i+1
    return grid

# BMPS Run

In [10]:
train_envs[21].reset()
trace = run_env(bo_pol, train_envs[21])
trace

{'actions': [1, 5, 9, 13, 10, 14, 11, 17, 21, 28],
 'ground_truth': array([6.232, 4.143, 6.703, 8.741, 7.313, 5.916, 3.641, 4.641, 8.306, 8.402, 6.132, 8.566, 4.376, 6.839, 6.25 , 8.518, 4.638, 7.33 , 6.62 , 1.532, 8.28 , 1.173, 3.232, 3.266, 2.941, 2.091, 8.289,
        7.248]),
 'observations': 9,
 'util': 7.010738263987953}

In [11]:
train_envs[21].dist

array([0.18, 0.36, 0.24, 0.22])

In [12]:
train_envs[21].grid()

array([[TNorm(5.00, 2.99,[0.01,9.99]), 4.14326783014364, TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99])],
       [TNorm(5.00, 2.99,[0.01,9.99]), 5.916257672589262, TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99])],
       [TNorm(5.00, 2.99,[0.01,9.99]), 8.401830218607623, 6.131778712164349, 8.56569315622621],
       [TNorm(5.00, 2.99,[0.01,9.99]), 6.8385177101359025, 6.250219553552345, TNorm(5.00, 2.99,[0.01,9.99])],
       [TNorm(5.00, 2.99,[0.01,9.99]), 7.329814725724805, TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99])],
       [TNorm(5.00, 2.99,[0.01,9.99]), 1.1731174611429962, TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99])],
       [TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99])]], dtype=object)

In [13]:
p_grid(train_envs[21],trace['actions'])

array([[0.18, 0.36, 0.24, 0.22],
       [0.  , 1.  , 0.  , 0.  ],
       [0.  , 2.  , 0.  , 0.  ],
       [0.  , 3.  , 5.  , 7.  ],
       [0.  , 4.  , 6.  , 0.  ],
       [0.  , 8.  , 0.  , 0.  ],
       [0.  , 9.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ]])

# DC Run

In [14]:
train_envs[21].reset()
trace = run_dc(train_envs[21])
trace

{'actions': [5, 6, 1, 9, 10, 11, 8, 28],
 'ground_truth': array([6.232, 4.143, 6.703, 8.741, 7.313, 5.916, 3.641, 4.641, 8.306, 8.402, 6.132, 8.566, 4.376, 6.839, 6.25 , 8.518, 4.638, 7.33 , 6.62 , 1.532, 8.28 , 1.173, 3.232, 3.266, 2.941, 2.091, 8.289,
        7.248]),
 'observations': 7,
 'options': [(1, 1), (1, 1), (0, 1), (2, 1), (2, 1), (2, 1), (2, 1), (-99, 1)],
 'util': 7.665733355700672}

In [15]:
train_envs[21].dist

array([0.18, 0.36, 0.24, 0.22])

In [16]:
train_envs[21].grid()

array([[TNorm(5.00, 2.99,[0.01,9.99]), 4.14326783014364, TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99])],
       [TNorm(5.00, 2.99,[0.01,9.99]), 5.916257672589262, 3.6407289514742835, TNorm(5.00, 2.99,[0.01,9.99])],
       [8.30552828729288, 8.401830218607623, 6.131778712164349, 8.56569315622621],
       [TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99])],
       [TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99])],
       [TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99])],
       [TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99]), TNorm(5.00, 2.99,[0.01,9.99])]], dtype=object)

In [17]:
p_grid(train_envs[21],trace['actions'])

array([[0.18, 0.36, 0.24, 0.22],
       [0.  , 3.  , 0.  , 0.  ],
       [0.  , 1.  , 2.  , 0.  ],
       [7.  , 4.  , 5.  , 6.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ]])

# Parsing

In [18]:
def make_env(gambles=7, cost=.01, ground_truth=False, dist=hd_dist(3), stakes = 'high'):
    reward = TruncatedNormal((9.99+0.01)/2, 0.3*(9.99-0.01),0.01,9.99) if stakes == 'high' else Normal((0.25+0.01)/2, 0.3*(0.25-0.01),0.01,0.25)
    return OldMouselabEnv(gambles, dist, reward, cost, ground_truth = ground_truth) 

In [19]:
env = train_envs[21]

In [20]:
env2 = make_env(ground_truth=env.ground_truth,dist=env.dist)

In [21]:
env.ground_truth

array([6.232, 4.143, 6.703, 8.741, 7.313, 5.916, 3.641, 4.641, 8.306, 8.402, 6.132, 8.566, 4.376, 6.839, 6.25 , 8.518, 4.638, 7.33 , 6.62 , 1.532, 8.28 , 1.173, 3.232, 3.266, 2.941, 2.091, 8.289,
       7.248])

In [22]:
env2.ground_truth

array([6.232, 4.143, 6.703, 8.741, 7.313, 5.916, 3.641, 4.641, 8.306, 8.402, 6.132, 8.566, 4.376, 6.839, 6.25 , 8.518, 4.638, 7.33 , 6.62 , 1.532, 8.28 , 1.173, 3.232, 3.266, 2.941, 2.091, 8.289,
       7.248])

In [23]:
trace = run_env(bo_pol, env)
trace

{'actions': [1, 5, 9, 13, 10, 14, 11, 17, 21, 28],
 'ground_truth': array([6.232, 4.143, 6.703, 8.741, 7.313, 5.916, 3.641, 4.641, 8.306, 8.402, 6.132, 8.566, 4.376, 6.839, 6.25 , 8.518, 4.638, 7.33 , 6.62 , 1.532, 8.28 , 1.173, 3.232, 3.266, 2.941, 2.091, 8.289,
        7.248]),
 'observations': 9,
 'util': 7.010738263987953}

In [24]:
trace = run_env(bo_pol, env2)
trace

{'actions': [1, 5, 9, 13, 10, 14, 11, 17, 21, 25, 18, 19, 15, 12, 8, 28],
 'ground_truth': array([6.232, 4.143, 6.703, 8.741, 7.313, 5.916, 3.641, 4.641, 8.306, 8.402, 6.132, 8.566, 4.376, 6.839, 6.25 , 8.518, 4.638, 7.33 , 6.62 , 1.532, 8.28 , 1.173, 3.232, 3.266, 2.941, 2.091, 8.289,
        7.248]),
 'observations': 15,
 'util': 7.7257333557006715}

In [91]:
def wrap_po2(env,click_sequence,p_rand=0):
    memo = dict()
    def parse_options_clean(init_state,dist,cost,stakes,pre_acts,click_sequence):
        if click_sequence == []: 
            return True, [[]], [1]
        
        if (tuple(pre_acts),tuple(click_sequence)) in memo:       
            return memo[(tuple(pre_acts),tuple(click_sequence))]   
        
        envc = make_env(gambles=int(len(init_state)/len(dist)), ground_truth=init_state, cost=cost, dist=dist, stakes=stakes)
        envc.reset()
        for a in pre_acts:
            envc._step(a)

        option_seqs = []
        likelihoods = []
        done = False
        options, option_insts, option_utils, n_available_clicks,_,_,_ = get_all_options(envc)
        
        for i,j in product(range(1,min(len(dist),len(click_sequence))+1),range(len(options))):
            option = options[j]
            if option_utils[j] == np.max(option_utils) or option == (-1,1):
                n_insts = len(option_insts[option])
                alpha = 1.0 if option == (-1,1) else 0.0
                beta = 1.0/np.sum(option_utils == np.max(option_utils)) if option_utils[j] == np.max(option_utils) else 0.0
                for inst in option_insts[option]:
                    if np.array_equal(click_sequence[:i],inst): 
                        will_done, remaining, rem_likelihoods = (parse_options_clean(init_state,dist,cost,stakes,pre_acts+click_sequence[:i],click_sequence[i:]))
                        done = done or will_done  
                        if done:
                            for k in range(len(remaining)):
                                option_seqs.append([option]+remaining[k]) 
                                l_opt_seq = (1-p_rand)*beta + p_rand*alpha
                                likelihoods.append(l_opt_seq*rem_likelihoods[k]/n_insts)               
        memo[(tuple(pre_acts),tuple(click_sequence))] = done, option_seqs, likelihoods
        return done, option_seqs, likelihoods
    
    stakes = 'high' if env.reward.mu == 5.0 else 'low'
    if click_sequence == [] or click_sequence[-1] != env.term_action:
        click_sequence = click_sequence+[env.term_action]
    return parse_options_clean(env.ground_truth,env.dist,env.cost,stakes,[],click_sequence)

In [92]:
env = OldMouselabEnv(4, ld_dist(3), reward, cost)

In [93]:
env.reset()
trace = run_dc(env)
click_seq = trace['actions']
click_seq

[1, 0, 2, 12]

In [94]:
trace['options']

[(0, 1), (0, 1), (0, 1), (-99, 1)]

In [95]:
a,b,c = wrap_po(env,click_seq,p_rand=0)

In [96]:
b

[[(0, 1), (0, 1), (0, 1), (-1, 1)],
 [(0, 1), (0, 1), (0, 1), (-99, 1)],
 [(0, 1), (0, 1), (-1, 1), (-1, 1)],
 [(0, 1), (0, 1), (-1, 1), (-99, 1)],
 [(0, 1), (-1, 1), (0, 1), (-1, 1)],
 [(0, 1), (-1, 1), (0, 1), (-99, 1)],
 [(0, 1), (-1, 1), (-1, 1), (-1, 1)],
 [(0, 1), (-1, 1), (-1, 1), (-99, 1)],
 [(0, 1), (0, 2), (-1, 1)],
 [(0, 1), (0, 2), (-99, 1)],
 [(-1, 1), (0, 1), (0, 1), (-1, 1)],
 [(-1, 1), (0, 1), (0, 1), (-99, 1)],
 [(-1, 1), (0, 1), (-1, 1), (-1, 1)],
 [(-1, 1), (0, 1), (-1, 1), (-99, 1)],
 [(-1, 1), (-1, 1), (0, 1), (-1, 1)],
 [(-1, 1), (-1, 1), (0, 1), (-99, 1)],
 [(-1, 1), (-1, 1), (-1, 1), (-1, 1)],
 [(-1, 1), (-1, 1), (-1, 1), (-99, 1)],
 [(-1, 1), (0, 2), (-1, 1)],
 [(-1, 1), (0, 2), (-99, 1)],
 [(0, 2), (0, 1), (-1, 1)],
 [(0, 2), (0, 1), (-99, 1)],
 [(0, 2), (-1, 1), (-1, 1)],
 [(0, 2), (-1, 1), (-99, 1)],
 [(0, 3), (-1, 1)],
 [(0, 3), (-99, 1)]]

In [97]:
list(zip(list(np.array(b)[np.array(c)!=0]),np.array(c)[np.array(c)!=0]))

[([(0, 1), (0, 1), (0, 1), (-99, 1)], 0.25)]

In [98]:
env.reset()
a,b,c = wrap_po2(env,click_seq,p_rand=0)

In [99]:
b

[[(0, 1), (0, 1), (0, 1), (-1, 1)],
 [(0, 1), (0, 1), (0, 1), (-99, 1)],
 [(0, 1), (0, 1), (-1, 1), (-1, 1)],
 [(0, 1), (0, 1), (-1, 1), (-99, 1)],
 [(0, 1), (-1, 1), (0, 1), (-1, 1)],
 [(0, 1), (-1, 1), (0, 1), (-99, 1)],
 [(0, 1), (-1, 1), (-1, 1), (-1, 1)],
 [(0, 1), (-1, 1), (-1, 1), (-99, 1)],
 [(-1, 1), (0, 1), (0, 1), (-1, 1)],
 [(-1, 1), (0, 1), (0, 1), (-99, 1)],
 [(-1, 1), (0, 1), (-1, 1), (-1, 1)],
 [(-1, 1), (0, 1), (-1, 1), (-99, 1)],
 [(-1, 1), (-1, 1), (0, 1), (-1, 1)],
 [(-1, 1), (-1, 1), (0, 1), (-99, 1)],
 [(-1, 1), (-1, 1), (-1, 1), (-1, 1)],
 [(-1, 1), (-1, 1), (-1, 1), (-99, 1)]]

In [100]:
list(zip(list(np.array(b)[np.array(c)!=0]),np.array(c)[np.array(c)!=0]))

[(array([[  0,   1],
         [  0,   1],
         [  0,   1],
         [-99,   1]]), 0.25)]

In [103]:
def dc_log_likelihood(env, trial, p_error):
    done, option_seqs, likelihoods = wrap_po2(env,trial,p_rand=p_error)
    return np.log(np.sum(likelihoods))

In [104]:
def wrap_sum_ll(p, p_error):
    data = participants[p]
    return sum(dc_log_likelihood(env, trial, p_error)
                    for env,trial in data)

# Apply to Data

In [105]:
import json
df = pd.read_csv('../risky_choice/data/03242018/trials.csv', index_col=0)
for col in ['outcome_probs', 'ground_truth', 'clicks']:
    df[col] = df[col].apply(json.loads)

In [106]:
df

,clicks,decision,ground_truth,outcome_probs,reward_mu,reward_sigma,trial_index,workerid
0,[],1,"[0.16, 0.2, 0.16, 0.07, 0.1, 0.2, 0.12, 0.11, ...","[0.16, 0.19, 0.39, 0.26]",0.13,0.072,0,A2HSCKH5NKN5LP
1,[],6,"[0.08, 0.18, 0.14, 0.16, 0.03, 0.14, 0.01, 0.1...","[0.35, 0.27, 0.24, 0.14]",0.13,0.072,1,A2HSCKH5NKN5LP
2,[],4,"[0.11, 0.2, 0.17, 0.08, 0.18, 0.16, 0.12, 0.03...","[0.06, 0.05, 0.86, 0.03]",0.13,0.072,2,A2HSCKH5NKN5LP
3,[],4,"[0.06, 0.17, 0.19, 0.14, 0.07, 0.18, 0.15, 0.0...","[0.02, 0.9, 0.02, 0.06]",0.13,0.072,3,A2HSCKH5NKN5LP
4,[],5,"[0.09, 0.09, 0.11, 0.13, 0.2, 0.09, 0.24, 0.08...","[0.08, 0.9, 0.01, 0.01]",0.13,0.072,4,A2HSCKH5NKN5LP
5,[],2,"[0.03, 0.1, 0.02, 0.13, 0.1, 0.11, 0.16, 0.03,...","[0.01, 0.01, 0.07, 0.91]",0.13,0.072,5,A2HSCKH5NKN5LP
6,[],6,"[0.15, 0.08, 0.18, 0.01, 0.06, 0.2, 0.1, 0.2, ...","[0.3, 0.23, 0.25, 0.22]",0.13,0.072,6,A2HSCKH5NKN5LP
7,[],5,"[0.12, 0.22, 0.16, 0.13, 0.08, 0.05, 0.08, 0.1...","[0.32, 0.11, 0.33, 0.24]",0.13,0.072,7,A2HSCKH5NKN5LP
8,[],5,"[0.09, 0.12, 0.15, 0.16, 0.16, 0.02, 0.19, 0.0...","[0.28, 0.12, 0.37, 0.23]",0.13,0.072,8,A2HSCKH5NKN5LP
9,[15],4,"[0.09, 0.1, 0.09, 0.19, 0.16, 0.1, 0.05, 0.12,...","[0.04, 0.05, 0.06, 0.85]",0.13,0.072,9,A2HSCKH5NKN5LP


In [107]:
j = 0
participants = dict()
for i, row in df.iterrows():
    stakes = 'high' if env.reward.mu == 5.0 else 'low'
    n_gambles = int(len(row['ground_truth'])/len(row['outcome_probs']))
    env = make_env(gambles=n_gambles, ground_truth=row['ground_truth'], cost=0.01, dist=row['outcome_probs'], stakes=stakes)
    if row['workerid'] in participants.keys():
        participants[row['workerid']].append((env,row['clicks']))
    else:
        participants[row['workerid']] =[(env,row['clicks'])]

In [108]:
parts = list(participants.keys())

In [109]:
p=parts[0]

In [110]:
data = participants[p] 

In [111]:
data

[(<oldmouselab.OldMouselabEnv at 0x2b865fe62048>, [4, 5, 9, 17]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe624a8>, [17, 1, 9, 21, 25, 5, 13]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe624e0>,
  [24, 22, 17, 15, 9, 6, 3, 25, 26, 27]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe62550>,
  [0, 1, 2, 3, 9, 14, 19, 23, 27, 11, 7]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe625c0>, [2, 6, 10, 14, 18, 22, 26]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe62630>, [2, 6, 10, 14, 18, 22, 26]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe626a0>, [1, 5, 9, 13, 17, 21, 25]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe62710>, [1, 5, 9, 13, 17, 21, 25]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe62780>,
  [3, 7, 11, 15, 19, 23, 27, 1]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe627f0>,
  [3, 7, 11, 15, 9, 19, 23, 27, 17]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe62860>, [1, 5, 9, 17, 21, 13]),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe628d0>, []),
 (<oldmouselab.OldMouselabEnv at 0x2b865fe62940>, []),
 

In [112]:
import copy

In [221]:
sm = []
probses = []
for p in parts:
    data = participants[p]
    probs = []
    for episode in data:
        env, trial = copy.deepcopy(episode)
        trial = trial+[env.term_action]
        prob = 1
        for a in trial:
            prob*=1/len(list(env.actions(env._state)))
            env.step(a)
        probs.append(prob)
    probses.append(probs)
    sm.append(np.sum(np.log(probs,dtype=np.float128)))

In [222]:
sm

[-332.4004665602713603,
 -67.34591659972948082,
 -343.68618024081067533,
 -480.95552677007896472,
 -391.35046705343204446,
 -417.35393154640452923,
 -367.93190724044591824,
 -635.3214949781059893,
 -708.9244822352629602,
 -841.53727429893264456,
 -699.18599455266860304,
 -583.9709854891756673,
 -603.4433038722485905,
 -746.24334797154483145,
 -239.89854950250164269,
 -352.0706143019141642,
 -103.147971234964048556,
 -644.6258339384223339,
 -733.2865391824923943,
 -550.07902091676501355,
 -497.7825423546295161,
 -556.7745324234056228,
 -348.52075937464472974,
 -536.46259057272865284,
 -590.55440057366594336,
 -242.18651846163371363,
 -486.9929256879823016,
 -368.96539752421140995,
 -600.1383617151730343,
 -415.12751293373600733,
 -561.6177149619994624,
 -357.10712827797444627,
 -263.95304139989413705,
 -250.25927097980552882,
 -264.48320406059489085,
 -474.68551317219129948,
 -509.54985533763940972,
 -90.411176225153433494,
 -775.3809935864289914,
 -448.99137192329128476,
 -148.72283509

In [223]:
ll_rand = np.sum(sm)

In [116]:
sm = []
for p in parts:
    sll = wrap_sum_ll(p, 1)
    print(sll)
    sm.append(sll)

-332.40046656027124
-67.34591659972948
-343.6861802408107
-480.9555267700791
-391.350467053432
-417.3539315464046
-367.9319072404459
-635.3214949781059
-708.924482235263
-841.537274298933
-699.1859945526686
-583.9709854891757
-603.4433038722487
-746.2433479715447
-239.89854950250165
-352.0706143019142
-103.14797123496405
-644.6258339384224
-733.2865391824923
-550.0790209167651
-497.7825423546295
-556.7745324234055
-348.5207593746448
-536.4625905727287
-590.5544005736659
-242.18651846163363
-486.9929256879824
-368.96539752421137
-600.1383617151731
-415.12751293373606
-561.6177149619995
-357.10712827797437
-263.9530413998941
-250.25927097980554
-264.48320406059486
-474.68551317219124
-509.5498553376394
-90.41117622515344
-775.380993586429
-448.9913719232913
-148.7228350990926
-512.8297906614781
-199.06219041377048
-363.1639561229767
-362.6437923369706
-569.7695169611687
-342.2103484789722
-67.34591659972948
-730.5628664023877
-212.8636496194404
-241.262168028117
-500.5776431552416
-431.5

In [117]:
np.sum(sm)

-44586.332420088125

# MLE Analysis

In [203]:
def mle(data):
# You can adjust the temperature bounds if you think the MLE
# is not in the bounds below. Don't change the p_error bounds.
    bounds = [
        (0, 0.5)  # p_error
    ]
    def loss(x):
        p_error = x
        return -sum(dc_log_likelihood(env, trial, p_error)
                    for env, trial in data)
    res = scipy.optimize.minimize(loss, (.01,),bounds = bounds,
                                 options={'maxiter':500,'disp':True})
    p_error = res.x
    return {'p_error': res.x[0], 'logp': -res.fun}

In [204]:
p = parts[0]

In [205]:
p

'A2YAYHZYI7M3HD'

In [206]:
def mle_wrap(p):
    try:
        d = np.load('data/mles/ll4_'+p+'.npy').item()
    except:     
        print(p)
        d = mle(participants[p])
        d['participant'] = p
        np.save('data/mles/ll4_'+p,d)
    print(d)
    return d

In [207]:
mle_wrap(p)

{'p_error': 0.5, 'participant': 'A2YAYHZYI7M3HD', 'logp': -335.7967183212359}


{'logp': -335.7967183212359, 'p_error': 0.5, 'participant': 'A2YAYHZYI7M3HD'}

In [208]:
mles = Parallel(n_jobs=100)(delayed(mle_wrap)(p) for p in parts)

{'p_error': 0.5, 'participant': 'A2YAYHZYI7M3HD', 'logp': -335.7967183212359}
{'p_error': 0.5, 'participant': 'AT7THGIGM9XN5', 'logp': -294.5516756829117}
{'p_error': 0.5, 'participant': 'A3MD34XEB4H6JF', 'logp': -456.2908309896481}
{'p_error': 0.5, 'participant': 'A1YSYI926BBOHW', 'logp': -81.2088602109284}
{'p_error': 0.5, 'participant': 'A3SOJWB6AZWZV9', 'logp': -350.3850301698782}
{'p_error': 0.5, 'participant': 'AUOLR6JOD7STS', 'logp': -376.14247690067907}
{'p_error': 0.4634331250840182, 'participant': 'A353STSNL3BYS6', 'logp': -300.7908245023345}
{'p_error': 0.5, 'participant': 'A1BAEJW3MDJ7FQ', 'logp': -564.3162397899449}
{'p_error': 0.5, 'participant': 'A1MJVTR0PCKBWW', 'logp': -494.9951843383284}
{'p_error': 0.5, 'participant': 'A3FOL78K2EQYVN', 'logp': -677.4745558023182}
{'p_error': 0.4204234387855232, 'participant': 'A2A7O6KYJ9GS8A', 'logp': -485.6583266890473}
{'p_error': 0.5, 'participant': 'A3JOUIM3CDC1DZ', 'logp': -577.8453956612015}
{'p_error': 0.5, 'participant': 'A2G

In [209]:
ll_dc = np.sum([p['logp'] for p in mles])

# Model Comparison

In [216]:
obses = []
for p in parts:
    data = participants[p]
    obssm = 0
    for env,trial in data:
        obssm+= len(trial)+1
    obses.append(obssm)

In [218]:
np.sum(obses)

14710

## DC Statistics

### Overall

In [224]:
ll_dc

-40415.25616479107

In [219]:
bic_dc = -2*ll_dc + len(mles) * (np.log(np.sum(obses))-np.log(2*np.pi))

In [220]:
bic_dc

81606.35290430076

In [228]:
aic_dc = -2*ll_dc + 2*len(mles)

In [229]:
aic_dc

81030.51232958214

### Individual Statistics

In [233]:
ll_dcs = [p['logp'] for p in mles]

In [245]:
parts == [p['participant'] for p in mles]

True

In [239]:
bic_dc_inds = -2*np.array(ll_dcs) + np.log(np.array(obses)) - np.log(2*np.pi)

In [240]:
bic_dc_inds

array([ 674.39 ,  163.576,  591.929,  915.748,  755.235,  703.792,  604.471,  974.777, 1358.625, 1680.795, 1159.358, 1132.093,  993.405, 1704.564,  438.018,  578.321,  207.132, 1110.626, 1597.459,
        883.799,  760.471,  915.713,  608.123, 1038.369, 1043.631,  436.195,  812.062,  676.39 , 1253.585,  732.789, 1008.741,  515.108,  454.503,  525.014,  492.371,  740.404,  835.119,  210.916,
       1757.169, 1026.224,  263.116,  876.726,  338.699,  652.806,  673.506, 1008.229,  517.998,  163.576, 1480.626,  387.064,  426.29 , 1099.698,  829.05 ,  464.234, 1006.302,  572.396,  494.714,
        525.202, 1491.032,  732.602,  596.588,  800.715,  967.287,  787.764,  179.772, 1262.205,  163.576,  831.938,  958.093,  943.698,  700.992, 1491.673,  198.829, 1049.321,  800.846, 1117.553,
        294.289,  454.666,  791.137, 1118.081,  919.957,  700.327,  876.07 ,  991.986,  949.312,  555.877,  673.095,  815.139,  169.431,  537.08 , 1699.127, 1749.888,  606.918,  973.041,  952.811,
        984.485

In [242]:
np.sum(bic_dc_inds)

81129.28166144942

In [246]:
aic_dc_inds = -2*np.array(ll_dcs) + 2

In [247]:
aic_dc_inds

array([ 673.593,  164.418,  591.103,  914.582,  754.285,  702.77 ,  603.582,  973.317, 1356.949, 1678.929, 1157.691, 1130.632,  991.99 , 1702.698,  437.565,  577.477,  207.536, 1109.107, 1595.631,
        882.489,  759.203,  914.386,  607.279, 1036.939, 1042.212,  435.742,  810.85 ,  675.5  , 1252.016,  731.767, 1007.364,  514.236,  453.958,  524.421,  491.827,  739.251,  833.87 ,  211.458,
       1755.273, 1025.113,  263.148,  875.476,  338.443,  651.925,  672.634, 1006.836,  517.182,  164.418, 1478.844,  386.743,  425.824, 1098.343,  827.998,  463.741, 1004.919,  571.48 ,  494.03 ,
        524.621, 1489.173,  731.572,  595.69 ,  799.49 ,  965.778,  786.814,  180.519, 1260.614,  164.418,  830.719,  956.777,  942.473,  699.917, 1490.06 ,  199.448, 1047.807,  799.779, 1116.128,
        294.084,  454.285,  790.146, 1116.507,  918.614,  699.283,  874.703,  990.462,  947.946,  554.996,  672.035,  813.947,  170.224,  536.374, 1697.215, 1747.958,  606.213,  971.785,  951.356,
        982.99 

In [248]:
np.sum(aic_dc_inds)

81030.51232958214

## Random Statistics

### Overall

In [225]:
ll_rand

-44586.33242008811676

In [226]:
bic_rand = -2*ll_rand

In [227]:
bic_rand

89172.66484017623352

In [230]:
aic_rand = -2*ll_rand

In [231]:
aic_rand

89172.66484017623352

### Individual Statistcs

In [250]:
ll_rand_inds = sm

In [251]:
bic_rand_inds = -2*np.array(ll_rand_inds)

In [252]:
bic_rand_inds

array([ 664.801,  134.692,  687.372,  961.911,  782.701,  834.708,  735.864, 1270.643, 1417.849, 1683.075, 1398.372, 1167.942, 1206.887, 1492.487,  479.797,  704.141,  206.296, 1289.252, 1466.573,
       1100.158,  995.565, 1113.549,  697.042, 1072.925, 1181.109,  484.373,  973.986,  737.931, 1200.277,  830.255, 1123.235,  714.214,  527.906,  500.519,  528.966,  949.371, 1019.1  ,  180.822,
       1550.762,  897.983,  297.446, 1025.66 ,  398.124,  726.328,  725.288, 1139.539,  684.421,  134.692, 1461.126,  425.727,  482.524, 1001.155,  863.097,  503.294, 1151.201,  738.765,  599.206,
        539.8  , 1757.563,  831.762,  745.531,  940.967, 1318.812,  766.099,  148.021, 1390.002,  134.692,  975.44 , 1047.81 , 1004.995,  883.135, 1441.911,  167.258, 1317.817,  874.405, 1200.451,
        373.117,  392.486,  812.359, 1313.446, 1139.654,  847.257, 1142.786, 1302.684, 1088.457,  730.738,  868.916,  954.805,  141.356,  618.123, 1582.525, 1623.477,  611.396, 1036.96 , 1209.777,
       1167.399

In [256]:
aic_rand_inds = bic_rand_inds

## Comparing Statistics

In [253]:
np.sum(bic_rand_inds > bic_dc_inds)

76

In [254]:
np.sum(bic_dc_inds > bic_rand_inds)

24

In [257]:
np.sum(aic_rand_inds > aic_dc_inds)

76

In [258]:
np.sum(aic_rand_inds < aic_dc_inds)

24

In [262]:
np.array_equal(aic_rand_inds > aic_dc_inds,bic_rand_inds > bic_dc_inds)

True

In [264]:
eps = [p['p_error'] for p in mles]

In [265]:
np.min(eps)

0.3371482885051445

In [266]:
np.max(eps)

0.5

In [271]:
np.exp(np.min(ll_dcs))

0.0

In [274]:
np.exp(np.max(ll_dcs))

5.388155598576241e-36